In [1]:
%reload_ext autoreload
%autoreload 2
from cudants.io.image import Image, BatchedImages
from cudants.registration.rigid import RigidRegistration
from cudants.registration.affine import AffineRegistration
import torch
from torch.optim import SGD, Adam
from torch.nn import functional as F
from torch import nn

In [2]:
# img1 = Image.load_file('/data/BRATS2021/training/BraTS2021_00598/BraTS2021_00598_t1.nii.gz')
# img2 = Image.load_file('/data/BRATS2021/training/BraTS2021_00597/BraTS2021_00597_t1.nii.gz')

# img2 = Image.load_file('/data/BRATS2021/training/BraTS2021_01100/BraTS2021_01100_t1.nii.gz')
# img2 = Image.load_file('/data/BRATS2021/training/BraTS2021_00262/BraTS2021_00262_t1.nii.gz')
# img2 = Image.load_file('/data/BRATS2021/training/BraTS2021_00403/BraTS2021_00403_t1.nii.gz')

# img1 = Image.load_file("images/atlas_2mm_1000_3.nii.gz")
# img2 = Image.load_file("images/atlas_2mm_1001_3.nii.gz")

img1 = Image.load_file("testing/atlas_2mm_1000_3.nii.gz")
img2 = Image.load_file("testing/reslice_affine_fx_1000_mv_1001.nii.gz")

In [3]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interact, interactive, IntSlider, Layout

def browse_image_slices(image_3d):
    # Infer the dimensions of image
    if len(image_3d.shape) == 3:
        dim_x, dim_y, dim_z = image_3d.shape
        color = False
    else:
        dim_x, dim_y, dim_z = image_3d.shape[:-1]
        color = True
    # Define a function to visualize the slices
    def plot_slice(x, y, z):
        fig, ax = plt.subplots(1, 3, figsize=(15, 5))

        ax[0].imshow(image_3d[x, :, :], cmap='gray' if not color else None)
        ax[0].set_title('Slice at X = %d' % x)

        ax[1].imshow(image_3d[:, y, :], cmap='gray' if not color else None)
        ax[1].set_title('Slice at Y = %d' % y)

        ax[2].imshow(image_3d[:, :, z], cmap='gray' if not color else None)
        ax[2].set_title('Slice at Z = %d' % z)

        plt.show()

    # Use interact to create the UI with sliders
    interact(plot_slice,
             x=IntSlider(min=0, max=dim_x-1, value=dim_x//2, layout=Layout(width='600px')),
             y=IntSlider(min=0, max=dim_y-1, value=dim_y//2, layout=Layout(width='600px')),
             z=IntSlider(min=0, max=dim_z-1, value=dim_z//2, layout=Layout(width='600px')))

In [4]:
# Test the function
browse_image_slices(img1.array[0, 0].data.cpu().numpy())

interactive(children=(IntSlider(value=64, description='x', layout=Layout(width='600px'), max=127), IntSlider(v…

In [5]:
# Test the function
browse_image_slices(img2.array[0, 0].data.cpu().numpy())

interactive(children=(IntSlider(value=64, description='x', layout=Layout(width='600px'), max=127), IntSlider(v…

In [6]:
from cudants.registration.affine import AffineRegistration
fixed = BatchedImages([img1,])
moving = BatchedImages([img2,])
# transform = AffineRegistration([8, 4, 2, 1], [1000, 500, 250, 100], fixed, moving, \
transform = AffineRegistration([8, 4, 2, 1], [200, 100, 50, 20], fixed, moving, \
    loss_type='cc', optimizer='Adam', optimizer_lr=3e-4, optimizer_momentum=0.9)
moved = transform.optimize(save_transformed=True)

/home/rohitrango/code/CUDANTs/cudants/utils/imageutils.py:75: UserWarning: operator() sees varying value in profiling, ignoring and this should be handled by GUARD logic (Triggered internally at ../torch/csrc/jit/codegen/cuda/parser.cpp:3668.)
  image_smooth = separable_filtering(image, gaussians)
/home/rohitrango/code/CUDANTs/cudants/registration/affine.py:76: UserWarning: operator() profile_node %70 : int[] = prim::profile_ivalue[profile_failed="varying profile values"](%s.1)
 does not have profile information (Triggered internally at ../torch/csrc/jit/codegen/cuda/graph_fuser.cpp:105.)
  moving_image_blur = separable_filtering(moving_arrays, gaussians)
scale: 1, iter: 19/20, loss: -0.198471: 100%|██████████████████████████████████████████████████████████████████████████████████| 20/20 [00:00<00:00, 135.16it/s]


In [7]:
print(transform.get_affine_matrix())

tensor([[[ 0.9366,  0.0307, -0.0157,  0.1154],
         [-0.0628,  0.9473, -0.0607,  0.1103],
         [-0.0035,  0.0027,  1.0137,  0.1051],
         [ 0.0000,  0.0000,  0.0000,  1.0000]]], device='cuda:0',
       grad_fn=<CatBackward0>)


In [8]:
browse_image_slices(moved[-1][0, 0].data.cpu().numpy() - img1.array[0, 0].data.cpu().numpy())
browse_image_slices(moved[-1][0, 0].data.cpu().numpy())
browse_image_slices(img1.array[0, 0].data.cpu().numpy())

interactive(children=(IntSlider(value=64, description='x', layout=Layout(width='600px'), max=127), IntSlider(v…

interactive(children=(IntSlider(value=64, description='x', layout=Layout(width='600px'), max=127), IntSlider(v…

interactive(children=(IntSlider(value=64, description='x', layout=Layout(width='600px'), max=127), IntSlider(v…

In [9]:
browse_image_slices(moved[-1][0, 0].data.cpu().numpy() - img2.array[0, 0].data.cpu().numpy())

interactive(children=(IntSlider(value=64, description='x', layout=Layout(width='600px'), max=127), IntSlider(v…

In [57]:
from cudants.registration.syn import SyNRegistration
reg = SyNRegistration([4, 2, 1], [200, 100, 50], fixed, moving, 
                        integrator_n=10, init_affine=transform.get_affine_matrix().detach(),
#                          optimizer='SGD', optimizer_lr=1, optimizer_momentum=0.95,
                         cc_kernel_size=3,
                         optimizer='Adam', optimizer_lr=1e-4, optimizer_params={},
                         smooth_grad_sigma=0.25, smooth_warp_sigma=0.25)
saved_deformable = reg.optimize(save_transformed=True)

scale: 1, iter: 49/50, loss: -0.337355: 100%|███████████████████████████████████████████████████████████████████████████████████| 50/50 [00:01<00:00, 30.77it/s]


In [58]:
# browse_image_slices(saved_deformable[-1][0, 0].data.cpu().numpy()- moved[-1][0, 0].data.cpu().numpy())
browse_image_slices(saved_deformable[-1][0, 0].data.cpu().numpy())
browse_image_slices(img1.array[0, 0].data.cpu().numpy())
browse_image_slices(img2.array[0, 0].data.cpu().numpy())

interactive(children=(IntSlider(value=64, description='x', layout=Layout(width='600px'), max=127), IntSlider(v…

interactive(children=(IntSlider(value=64, description='x', layout=Layout(width='600px'), max=127), IntSlider(v…

interactive(children=(IntSlider(value=64, description='x', layout=Layout(width='600px'), max=127), IntSlider(v…

# Greedy registration

In [51]:
from cudants.registration.deformation.geodesic import GeodesicShooting
from cudants.registration.greedy import GreedyRegistration
reg = GreedyRegistration([4, 2, 1], [200, 100, 50], fixed, moving, 
                        integrator_n=10, init_affine=transform.get_affine_matrix().detach(),
#                          optimizer='SGD', optimizer_lr=1, optimizer_momentum=0.0,
                        optimizer='Adam', optimizer_lr=1e-2, optimizer_params={},
                         cc_kernel_size = 5,
                         smooth_grad_sigma=0.25, smooth_warp_sigma=0.25)
saved_deformable = reg.optimize(save_transformed=True)

Adam {}


scale: 1, iter: 49/50, loss: -0.505194: 100%|███████████████████████████████████████████████████████████████████████████████████| 50/50 [00:00<00:00, 51.53it/s]


In [52]:
# browse_image_slices(saved_deformable[-1][0, 0].data.cpu().numpy()- moved[-1][0, 0].data.cpu().numpy())
browse_image_slices(saved_deformable[-1][0, 0].data.cpu().numpy())
browse_image_slices(img1.array[0, 0].data.cpu().numpy())
browse_image_slices(img2.array[0, 0].data.cpu().numpy())

interactive(children=(IntSlider(value=64, description='x', layout=Layout(width='600px'), max=127), IntSlider(v…

interactive(children=(IntSlider(value=64, description='x', layout=Layout(width='600px'), max=127), IntSlider(v…

interactive(children=(IntSlider(value=64, description='x', layout=Layout(width='600px'), max=127), IntSlider(v…

# Log demons

In [ ]:
from cudants.registration.logdemons import LogDemonsRegistration
reg = LogDemonsRegistration([4, 2, 1], [50, 50, 50], fixed, moving, 
# reg = LogDemonsRegistration([1], [50], fixed, moving,
                eps_prime=0.25, init_affine=transform.get_affine_matrix().detach(),  
                optical_flow_method='gauss-newton', update_sigma=0.1,
                                optical_flow_sigma=0.1, use_lie_bracket=False, symmetric=False)
saved_deformable = reg.optimize(save_transformed=True)

In [ ]:
browse_image_slices(saved_deformable[-1][0, 0].data.cpu().numpy()- moved[-1][0, 0].data.cpu().numpy())
browse_image_slices(saved_deformable[-1][0, 0].data.cpu().numpy())
browse_image_slices(img1.array[0, 0].data.cpu().numpy())
browse_image_slices(img2.array[0, 0].data.cpu().numpy())
# browse_image_slices(moved[-1][0, 0].data.cpu().numpy())

In [ ]:
# browse_image_slices(saved_deformable[-1][0, 0].data.cpu().numpy()- moved[-1][0, 0].data.cpu().numpy())
# browse_image_slices(saved_deformable[-1][0, 0].data.cpu().numpy())
# browse_image_slices(img1.array[0, 0].data.cpu().numpy())
# browse_image_slices(img2.array[0, 0].data.cpu().numpy())
# browse_image_slices(moved[-1][0, 0].data.cpu().numpy())

## Check Optical Flow

In [ ]:
browse_image_slices(reg.velocity_fields[-1][0].data.cpu().numpy() - reg.velocity_fields[-1].min().item())